Assignment 3: Caitlin Box

In [99]:
import pygame
import moderngl
from math import cos, sin, radians
import numpy as np

In [100]:
WIDTH = 600
HEIGHT = 600
aspect_ratio = WIDTH / HEIGHT
pygame.init()

(5, 0)

In [101]:
pygame.display.set_mode((HEIGHT, WIDTH), flags= pygame.OPENGL | pygame.DOUBLEBUF | pygame.RESIZABLE)
pygame.display.set_caption(title="Assignment 4: Caitlin Box")
gl = moderngl.get_context()

In [102]:
# two different triangles to make diamond 
position_data = [
    #x    y 
    0.0, 0.8,
    0.8, 0.0,
    -0.8, 0.0,

    0.0, -0.8,
    0.8, 0.0,
    -0.8, 0.0
]
triangle_vertex_positins = np.array(position_data).astype("float32")
triangle_position_buffer = gl.buffer(triangle_vertex_positins)

Fragment Shader 
    inColor will allow us to assignment different colors to different objects 

In [103]:
fragment_shader_code = '''
#version 330 core

out vec4 color; 
uniform vec3 inColor; 

void main(){
    color = vec4(inColor, 0);
}
'''

Vertex Shader 
    The logic for what we need for the different shapes, having uniform will allow us to assign different measurments as needed 

In [104]:
vertex_shader_code = '''
#version 330 core

layout (location = 0) in vec2 inPosition;

uniform float scale;
uniform float distance;
uniform mat2 M;

void main() {
    vec2 d_vector = distance * vec2(1, 0); 
    vec2 P = inPosition * scale + d_vector;

    gl_Position = vec4(M * P, 0.0, 1.0); 
}
'''
vertex_shader_code_line = '''
#version 330 core

uniform float distance;
uniform mat2 M;

void main() {
    vec2 P = vec2(0);
    if (gl_VertexID > 0) {
        // Match diamond's displacement vector
        vec2 displacement_vector = distance * vec2(1, 0);
        P += displacement_vector;
    }

    gl_Position = vec4(M * P, 0.0, 1.0);
}
'''

In [105]:
program = gl.program(
    vertex_shader = vertex_shader_code,
    fragment_shader = fragment_shader_code
)
program_line = gl.program(
    vertex_shader = vertex_shader_code_line,
    fragment_shader = fragment_shader_code
)

In [106]:
renderable = gl.vertex_array(program,
    [(triangle_position_buffer, "2f", "inPosition")])

line_renderable = gl.vertex_array(program_line, [])

In [107]:
running = True
clock = pygame.time.Clock()
angle = 270

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT or (event.type == pygame.KEYDOWN and event.key==27):
            running = False
        elif event.type == pygame.VIDEORESIZE:
            WIDTH, HEIGHT = event.size  # update global variables!
            aspect_ratio = WIDTH / HEIGHT
            screen = pygame.display.set_mode((WIDTH, HEIGHT), pygame.OPENGL | pygame.DOUBLEBUF | pygame.RESIZABLE)

    gl.clear(0.5, 0.5, 0) # background color 

    aspect_ratio = WIDTH / HEIGHT
    angle_rad = radians(-angle)

    M = np.array([
        [cos(angle_rad) / aspect_ratio, -sin(angle_rad) / aspect_ratio],
        [sin(angle_rad),                 cos(angle_rad)]
    ], dtype='f4')

    program['M'].write(M.T.tobytes())  
    program_line['M'].write(M.T.tobytes())
    
    # first diamond 
    # scale changes diamond size 
    program ['scale'] = 0.1
    program ['distance'] = 0
    program['inColor'] = [1,1,0]
    renderable.render(moderngl.TRIANGLES, vertices = 6) 

    # changing location of the second diamond 
    program ['distance'] = 0.8
    renderable.render(moderngl.TRIANGLES, vertices = 6) 

    # creating the line 
    program_line ['distance'] = 0.8
    line_renderable.render(moderngl.LINES, vertices = 2)

    pygame.display.flip()
    clock.tick(60)

    # make the angle change so the shapes will roate on screen 
    angle = angle + 0.1
    if angle > 360:
        angle = 0

pygame.quit()